# Function calling

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers.openai_tools import JsonOutputToolsParser
from langchain_core.pydantic_v1 import BaseModel, Field

### Chat

Previously used Chat LLM

```python
chat_llm = ChatOpenAI(
    base_url="http://localhost:8080/v1",
    api_key="dev",
    model="mistral",
    temperature=0,
    max_tokens=128,
)
```

### Function Calling

In [ ]:
def format_string(string, max_len=120):  
    words = string.split(" ")  
    lines = []  
    current_line = []  
    current_length = 0  

    for word in words:  
        if current_length + len(word) + len(current_line) > max_len:  
            lines.append(" ".join(current_line))  
            current_line = [word]  
            current_length = len(word)  
        else:  
            current_line.append(word)  
            current_length += len(word)  

    lines.append(" ".join(current_line))  

    return "\n".join(lines)

In [ ]:
class Multiply(BaseModel):
    """Multiply two integers together."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


class SendEmail(BaseModel):
    """Send an email to receiver."""

    receiver: str = Field(..., description="Email receiver")
    topic: str = Field(..., description="Email topic")
    content: str = Field(..., description="Short email content, max 3 lines. Sign the email as Adam")

In [ ]:
function_calling_llm = ChatOpenAI(
    base_url="http://localhost:8080/v1",
    api_key="dev",
    model="mistral-function-calling",
    temperature=0,
    max_tokens=256,
)
tools = [Multiply, SendEmail]
function_calling_llm = function_calling_llm.bind_tools(tools)
function_calling_chain = function_calling_llm | JsonOutputToolsParser()

In [ ]:
result = function_calling_chain.invoke("What is 3 * 12?")
result

In [ ]:
mul = Multiply(**result[0]['args']['input'])
mul.a * mul.b

In [ ]:
result = function_calling_chain.invoke("Write an email to orders@carparts.com to place an order for new tyres for my Mercedes S-class")
result

In [ ]:
email = SendEmail(**result[0]['args'])
print("SENDING EMAIL")
print()
print(f"Receiver: {email.receiver!r:<10}")
print(f"Topic: {email.topic!r:<10}")
print("-" * 80)
print()
print(format_string(email.content, max_len=80))

### Try to confuse it

In [ ]:
result = function_calling_chain.invoke("""Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod
tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris
nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore
eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit
anim id est laborum.
Write an email to orders@carparts.com to place an order for new tyres for my Mercedes S-class.
Sed ut perspiciatis unde omnis iste natus error sit voluptatem accusantium doloremque laudantium, totam rem aperiam,
eaque ipsa quae ab illo inventore veritatis et quasi architecto beatae vitae dicta sunt explicabo.""")
result